In [45]:
import os
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, make_scorer

from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier


import eli5

pd.set_option('max_columns', None)

In [4]:
def data_preprocess(df):
    return df

In [5]:
def generate_next_submission_fileid():
    files_found = []
    for file in os.listdir("../../data"):
        if file.startswith("water_pump_submission"):
            files_found.append(file[22:24])
    if not files_found:
        return '01'
    return f'{int(sorted(files_found).pop()) + 1 :02}'

In [6]:
def submission_file_error_counts(df):
    a = df['status_group'].value_counts().rename_axis('status_group').reset_index(name='predicted_count')
    a.insert(1,'target_count',[8110,5672,1068])
    a['count_error'] = a['predicted_count'] - a['target_count']
    display(a)

In [7]:
def create_submission_file(pipeline, X_test, filename_comment, print_errors=False):
    next_file_id = generate_next_submission_fileid()
    X_test_processed = data_preprocess(X_test)
    y_pred_values = pipeline.predict(X_test_processed)
    
    ids = X_test['id'].reset_index(drop=True)
    y_pred = pd.DataFrame(data=y_pred_values, columns=["status_group"])
    y_pred.insert(0, 'id', ids)
    
    if print_errors: submission_file_error_counts(y_pred)
    
    filename = f'../../data/water_pump_submission_{next_file_id}_{filename_comment}.csv'
    y_pred.to_csv(filename, index = False)
    
    return y_pred, filename

In [46]:
X_raw = pd.read_csv('../../data/training_set_values.csv')
y_raw = pd.read_csv('../../data/training_set_labels.csv')

In [47]:
X_test_raw = pd.read_csv('../../data/test_set_values.csv')
y_test = pd.read_csv('../../data/SubmissionFormat.csv')

In [48]:
df = X_raw.merge(y_raw, left_on = 'id', right_on = 'id')
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [11]:
print(df['status_group'].unique())

['functional' 'non functional' 'functional needs repair']


### Common Pipeline Prep

In [12]:
num_cols = X_raw.select_dtypes('number').columns.tolist()
num_cols.pop(0)
num_cols

['amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'region_code',
 'district_code',
 'population',
 'construction_year']

In [13]:
obj_cols = X_raw.select_dtypes('object').columns.tolist()
obj_cols

['date_recorded',
 'funder',
 'installer',
 'wpt_name',
 'basin',
 'subvillage',
 'region',
 'lga',
 'ward',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'scheme_name',
 'permit',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quality_group',
 'quantity',
 'quantity_group',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group']

In [14]:
bool_cols = [col for col in obj_cols if type(X_raw[col][0])==bool]
bool_cols

['public_meeting', 'permit']

In [15]:
cat_cols_raw = [col for col in obj_cols if col not in bool_cols]
cat_cols_raw

['date_recorded',
 'funder',
 'installer',
 'wpt_name',
 'basin',
 'subvillage',
 'region',
 'lga',
 'ward',
 'recorded_by',
 'scheme_management',
 'scheme_name',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quality_group',
 'quantity',
 'quantity_group',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group']

In [16]:
# limit categorical columns we use to those with 25 or less unique values
cat_cols = [a for a in obj_cols if len(X_raw[a].unique()) <= 25]
display(cat_cols)

['basin',
 'region',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'permit',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quality_group',
 'quantity',
 'quantity_group',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group']

In [17]:
# Drop cols per feature importance evaluation
drop_list = ['waterpoint_type_group', 
             'extraction_type_group', 'extraction_type_class',
             'source_type', 'source_class',
             'quality_group',
             'quantity_group'
            ]
cat_cols_pipe = [col for col in cat_cols if col not in drop_list]
cat_cols_pipe

['basin',
 'region',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'permit',
 'extraction_type',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quantity',
 'source',
 'waterpoint_type']

In [49]:
y = df['status_group']
X = df.drop(columns=['status_group'])
X['date_recorded'] = pd.to_datetime(X['date_recorded'])

# for a first effort, we'll convert bool cols to strings and one-hot encode
for col in bool_cols:
    X[col] = X[col].apply(lambda x: 'True' if x==True else ('False' if x==False else 'missing') )
    
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [50]:
model = RandomForestClassifier(random_state=42) #AdaBoostClassifier(random_state=42)

In [51]:
num_pipe = Pipeline([('imputer', SimpleImputer(strategy = 'median')),
                     ('scaler', StandardScaler())])

cat_pipe = Pipeline([('imputer', SimpleImputer(strategy = 'constant', fill_value = 'missing')),
                     ('encoder', OneHotEncoder(handle_unknown = 'ignore', sparse = False))])

In [52]:
preprocessor = ColumnTransformer(transformers = [('cat', cat_pipe, cat_cols_pipe),
                                                 ('num', num_pipe, num_cols)],
                                remainder = 'drop')

In [53]:
pipe = Pipeline([('preprocessor', preprocessor),
                 ('model', model)])

In [54]:
pipe.fit(X_train, y_train);

In [55]:
y_pred_values = pipe.predict(X_val)

### Feature Importance Eval

In [26]:
one_hot_cols = pipe.named_steps['preprocessor'].named_transformers_['cat'].named_steps['encoder'].get_feature_names(input_features=cat_cols_pipe)

In [27]:
one_hot_cols

array(['basin_Internal', 'basin_Lake Nyasa', 'basin_Lake Rukwa',
       'basin_Lake Tanganyika', 'basin_Lake Victoria', 'basin_Pangani',
       'basin_Rufiji', 'basin_Ruvuma / Southern Coast',
       'basin_Wami / Ruvu', 'region_Arusha', 'region_Dar es Salaam',
       'region_Dodoma', 'region_Iringa', 'region_Kagera', 'region_Kigoma',
       'region_Kilimanjaro', 'region_Lindi', 'region_Manyara',
       'region_Mara', 'region_Mbeya', 'region_Morogoro', 'region_Mtwara',
       'region_Mwanza', 'region_Pwani', 'region_Rukwa', 'region_Ruvuma',
       'region_Shinyanga', 'region_Singida', 'region_Tabora',
       'region_Tanga', 'public_meeting_False', 'public_meeting_True',
       'public_meeting_missing', 'recorded_by_GeoData Consultants Ltd',
       'scheme_management_Company', 'scheme_management_None',
       'scheme_management_Other', 'scheme_management_Parastatal',
       'scheme_management_Private operator', 'scheme_management_SWC',
       'scheme_management_Trust', 'scheme_managemen

In [28]:
all_cols_post_one_hot = num_cols.copy()
all_cols_post_one_hot.extend(one_hot_cols)
len(all_cols_post_one_hot)

138

In [29]:
eli5.explain_weights(pipe.named_steps['model'], top=50, feature_names=all_cols_post_one_hot)

Weight,Feature
0.1418 ± 0.0115,waterpoint_type_cattle trough
0.1408 ± 0.0123,waterpoint_type_communal standpipe
0.0700 ± 0.0654,payment_type_unknown
0.0679 ± 0.0083,source_unknown
0.0518 ± 0.0119,waterpoint_type_other
0.0468 ± 0.0064,waterpoint_type_improved spring
0.0306 ± 0.0474,source_shallow well
0.0279 ± 0.0268,water_quality_coloured
0.0262 ± 0.0439,extraction_type_afridev
0.0238 ± 0.0056,waterpoint_type_hand pump


In [30]:
y_val.unique().tolist()

['non functional', 'functional', 'functional needs repair']

In [31]:
print(classification_report(y_val, y_pred_values, target_names=y_val.unique().tolist()))

                         precision    recall  f1-score   support

         non functional       0.81      0.87      0.84      6457
             functional       0.49      0.36      0.42       851
functional needs repair       0.83      0.78      0.80      4572

               accuracy                           0.80     11880
              macro avg       0.71      0.67      0.69     11880
           weighted avg       0.79      0.80      0.80     11880



In [32]:
filename_comment = "ada_boost_1"
y_pred, filename = create_submission_file(pipe, X_test_raw, filename_comment, print_errors=False)

### Grid Search

In [60]:
grid_space = {'RFC' : {'model__n_estimators': [50, 100, 150, 200],
                       'model__max_features': ['auto', 'sqrt', 'log2'],
                       'model__max_depth': range(3, 10),
                      },
              'ADA' : {'model__n_estimators': [50, 100, 150, 200],
                       'model__learning_rate': [0.01, 0.1, 0.2, 0.3],
                       'model__algorithm': ['SAMME', 'SAMME.R'],
                      }
             }

In [61]:
models = {'RFC': RandomForestClassifier(random_state=42), 
          'ADA' : AdaBoostClassifier(random_state=42)
         }

In [62]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [63]:
scoring_list = []
for name, model in models.items():
    pipe = Pipeline([('preprocessor', preprocessor),
                 ('model', model)])
    grid_search = GridSearchCV(pipe, grid_space[name], cv=kfold, scoring = 'accuracy',
                              verbose=3, n_jobs=-1)
    grid_search.fit(X, y)
    scoring_list.append(grid_search)

Fitting 10 folds for each of 84 candidates, totalling 840 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/home/asa/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  2.7min


KeyboardInterrupt: 